In [6]:
# ! pip install -q tensorflow numpy pandas scikit-learn mlxtend dataprep
# ! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
# ! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 798kB 15.3MB/s 
     |████████████████████████████████| 8.6MB 26.3MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 5.5MB 39.2MB/s 
     |████████████████████████████████| 1.4MB 23.3MB/s 
     |████████████████████████████████| 512kB 44.4MB/s 
     |████████████████████████████████| 368kB 43.4MB/s 
     |████████████████████████████████| 2.2MB 33.8MB/s 
     |████████████████████████████████| 655kB 41.7MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
ERROR: panel 0.9.7 has requirement bokeh>=2.1, but you'll have bokeh 2.0.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 5.0.2 which is incompatible.
ERROR: distributed 2.22.0 has requirement clou

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import date
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [39]:
df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)
# df['Embedding'] = np.array([embedding[1][i].split() for i in range(1600)]).astype(float).tolist()

In [40]:
df.head()

,File,Engagement,Date,Manipulation,Visual,Text,Meme
0,0.jpg,11,2019-08-12,1,Salvini,satira cruda votaci.. saremo di parola il part...,1
1,1.jpg,2261,2019-08-21,0,0,"peccato per le accise, dai. c'eravamo quasi.",0
2,2.jpg,1327,2019-08-23,0,"Salvini, Di Maio","""che c'è ancora?"" ""aspetto un bambino""",1
3,3.jpg,18,2019-08-27,0,0,ora in onda speciale gr1 ascolta la diretta se...,0
4,4.jpg,617,2019-08-25,0,"Salvini, Di Maio",sta sciocchezza che sto pe fa e solo colpa tua,1


In [25]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [26]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [50]:
import pickle

df['Text'] = pickle.load(open('bert_embeddings.pkl', 'rb')).tolist()
df['Visual'] = list(ohe(create_map()))
df['Embedding'] = np.array([embedding[1][i].split() for i in range(1600)]).astype(float).tolist()

In [51]:
mms = MinMaxScaler()
df['Date'] = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))

In [52]:
df.head()

,File,Engagement,Date,Manipulation,Visual,Text,Meme,Embedding
0,0.jpg,11,0.768193,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.05149292200803757, 0.14016880095005035, -0....",1,"[0.2158726453781128, 0.7257009148597717, 0.163..."
1,1.jpg,2261,0.772530,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.14450794458389282, 0.03751036897301674, -0....",0,"[0.1284249722957611, 0.6184784770011902, 0.162..."
2,2.jpg,1327,0.773494,0,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.09801547229290009, 0.05487951263785362, 0....",1,"[0.5046870112419128, 2.1724114418029785, 1.703..."
3,3.jpg,18,0.775422,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.134096160531044, 0.06178298220038414, 0.349...",0,"[0.0, 0.2783641517162323, 0.11101881414651871,..."
4,4.jpg,617,0.774458,0,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.10299677401781082, 0.049986328929662704, -0...",1,"[0.47237470746040344, 2.103384256362915, 1.693..."


In [53]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

In [64]:
def make_array(lst):
  res = []
  for i in lst:
    res.append(np.array(i))
  return np.array(res)

In [67]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  
  labels = dataframe.pop('Meme')
  vis = dataframe.pop('Visual')
  img = dataframe.pop('Embedding')
  txt = dataframe.pop('Text')
  params = dict(dataframe)
  params['Visual'] = make_array(vis)
  params['Embedding'] = make_array(img)
  params['Text'] = make_array(txt)
  ds = tf.data.Dataset.from_tensor_slices((params, labels))
  if shuffle:   
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [68]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [69]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of engagements:', feature_batch['Text'])
  print('A batch of targets:', label_batch )

Every feature: ['File', 'Engagement', 'Date', 'Manipulation', 'Visual', 'Embedding', 'Text']
A batch of engagements: tf.Tensor(
[[-0.1465358  -0.0284294  -0.55777252 ...  0.24718685  0.1133495
   0.13520588]
 [ 0.00151352  0.10473099 -0.00500363 ...  0.10566957  0.14342795
  -0.07385524]
 [ 0.02012596  0.0483054   0.22587338 ...  0.27419341  0.17249916
  -0.03101728]
 ...
 [-0.09571594  0.07777996 -0.15873204 ...  0.17822236  0.07788669
  -0.10058031]
 [ 0.19642644 -0.01943769  0.52932936 ...  0.50859398  0.22497822
  -0.28269506]
 [-0.06950282 -0.00476638  0.54132372 ...  0.44549501  0.05188809
  -0.14478819]], shape=(32, 768), dtype=float64)
A batch of targets: tf.Tensor([1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 1 0], shape=(32,), dtype=int64)


In [70]:
feature_columns = []

In [71]:
for header in ['Engagement', 'Date', 'Manipulation']:
  feature_columns.append(tf.feature_column.numeric_column(header))

In [72]:
feature_columns.append(tf.feature_column.numeric_column(key='Embedding', shape=[2048,]))
feature_columns.append(tf.feature_column.numeric_column(key='Text', shape=[768,]))

In [73]:
visual = tf.feature_column.categorical_column_with_vocabulary_list('Visual', np.unique(df['Visual'].values))
vis_one_hot = tf.feature_column.indicator_column(visual)
feature_columns.append(vis_one_hot)

ValueError: ignored

In [74]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [75]:
print(feature_columns)

[NumericColumn(key='Engagement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Date', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Manipulation', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Embedding', shape=(2048,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Text', shape=(768,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [83]:
model = tf.keras.Sequential([feature_layer,
                               tf.keras.layers.Dense(1024, 
                                                     name='dense_1',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dropout(rate=0.2), ## Dropout to improve generalization performance
                               tf.keras.layers.Dense(2048, 
                                                     name='dense_2',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dropout(rate=0.2),
                               tf.keras.layers.Dense(1024, 
                                                     name='dense_3',
                                                     activation=tf.keras.activations.relu),
                               tf.keras.layers.Dropout(rate=0.2),
                               tf.keras.layers.Dense(1, name='output',
                                                     activation=tf.keras.activations.sigmoid)])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [84]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/50
32/32 [==============================] - 3s 88ms/step - loss: 0.8050 - binary_accuracy: 0.5068 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 2/50
32/32 [==============================] - 3s 89ms/step - loss: 0.8054 - binary_accuracy: 0.5078 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 3/50
32/32 [==============================] - 3s 84ms/step - loss: 0.8054 - binary_accuracy: 0.5078 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 4/50
32/32 [==============================] - 3s 92ms/step - loss: 0.8054 - binary_accuracy: 0.5078 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 5/50
32/32 [==============================] - 3s 85ms/step - loss: 0.8054 - binary_accuracy: 0.5078 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 6/50
32/32 [==============================] - 3s 97ms/step - loss: 0.8054 - binary_accuracy: 0.5078 - val_loss: 0.8328 - val_binary_accuracy: 0.4805
Epoch 7/50
32/32 [==============================] - 3s 84ms/step - los